# Kreiranje fiktivnih priča koristeći GPT-Neo (Making a fiction story generator using a GPT-Neo)

## 1. Uvod

GPT-Neo je serija jezičnih modela baziranih na transformatorima razvijenih od strane EleutherAI. Oni su trenirani na velikim količinama podataka kako bi naučili obrasce i strukture prirodnog jezika. Naziv "GPT" označava "Generative Pre-trained Transformer," što se odnosi na osnovnu arhitekturu, dok "Neo" označava novu ili unaprijeđenu verziju ovih modela. Kao i drugi GPT modeli, GPT-Neo je istreniran na ogromnoj količini tekstualnih podataka sa interneta, što mu omogućava da nauči obrasce i strukture prirodnog jezika. Ovo treniranje omogućava GPT-Neu da izvršava širok spektar zadataka vezanih za razumijevanje i generiranje prirodnog jezika, kao što su dopunjavanje teksta, prijevod, odgovaranje na pitanja i mnoge druge, sa impresivnom preciznošću i točnošću. Različite verzije GPT-Nea se mogu razlikovati po veličini modela, podacima za treniranje i fino podešavanje za specifične zadatke. 

Ja sam koristio GPT-Neo 125M zato što je to model EleutherAI-a sa najmanje parametara, i mogu ga stabilno pokretati na Paperspace platformi. Također, GPT-Neo je opširniji i bolje se ponaša u određenim slučajevima od GPT-3, koji ni nije open-source. Za fino podešavanje, koristio sam sljedeći [skup podataka](https://github.com/facebookresearch/fairseq/tree/main/examples/stories). Ovaj skup podataka sadrži 300.000 priča i promptova za te priče.Tim iz Facebook-a je sakupio priče i promptove sa Reddit-ovog [ WritingPrompts forum](https://www.reddit.com/r/WritingPrompts/), koje su korisnici pisali tokom 3 godine, i kombinirao ih u ovaj skup podataka. WritingPrompts je Reddit zajednica gdje korisnici inspiriraju jedni druge da pišu tako što postavljaju promptove ili upite, a drugi korisnici slobodno odgovaraju u obliku priča. Svaki upit može imati više priča. Promptovi pokrivaju širok spektar tema, dužina i detalja. Priče moraju sadržavati najmanje 30 riječi, izbjegavati vulgarnost i neprikladan sadržaj, i trebaju biti inspirirane odgovarajućim promptom. Kombinirao sam promptove sa odgovarajućim pričama i fino podesio GPT-Neo model koristeći Hugging Face Transformers biblioteku. 

U ovom notebook-u, koristio sam Hugging Face Trainer API za fino podešavanje mog modela i Paperspace Pro plan za svoje okruženje. Kao metriku sam koristio perpleksnost kako bih provjerio da li je fino podešavanje poboljšalo moj model.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import logging
from tqdm import tqdm
import math
import argparse
import os

In [ ]:
pip install --upgrade huggingface_hub

In [1]:
pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.3
    Uninstalling transformers-4.21.3:
      Successfully uninstalled transformers-4.21.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 19.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## 2. Priprema podataka

### 2.1 Preuzimanje skupa podataka i kombiniranje promptova sa pričama.

Nakon preuzimanja tekstualnih datoteka s ovog [linka](https://github.com/facebookresearch/fairseq/tree/main/examples/stories). Promptovi i priče su spremljeni u odvojene datoteke. Datoteka valid.wp_source sadrži navedene promptove, dok valid.wp_target sadrži odgovarajuće priče. Također, treba napomenuti da je skup podataka za obuku prilično velik.

Kako bih istovremeno unio i prompt i priču u GPT-Neo, spajam propmtove i priče u jednu zajedničku cjelinu. Kao rezultat toga, svaka linija unutar ove ujedinjene datoteke sadrži i prompt i odgovarajuću priču.

In [15]:
!cd examples/stories
!curl https://dl.fbaipublicfiles.com/fairseq/data/writingPrompts.tar.gz | tar xvzf -

/bin/bash: line 0: cd: examples/stories: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0writingPrompts/
writingPrompts/test.wp_source
writingPrompts/test.wp_target
  4  363M    4 15.9M    0     0  29.7M      0  0:00:12 --:--:--  0:00:12 29.6MwritingPrompts/README
writingPrompts/valid.wp_source
writingPrompts/valid.wp_target
writingPrompts/train.wp_target
 92  363M   92  336M    0     0  44.7M      0  0:00:08  0:00:07  0:00:01 47.0MwritingPrompts/train.wp_source
100  363M  100  363M    0     0  44.9M      0  0:00:08  0:00:08 --:--:-- 47.1M


In [8]:
DATAPATH='writingPrompts'
def combinetext(prompt, story):
    fp=open(os.path.join(DATAPATH,prompt),encoding='utf8')
    fs=open(os.path.join(DATAPATH,story),encoding='utf8')
    prompts=fp.readlines()
    stories=fs.readlines()
    assert len(prompts)==len(stories)
    combine=[]
    for i in range(len(prompts)):
        combine.append(prompts[i].rstrip()+' <sep> '+" ".join(stories[i].split()[:300]))
    return combine

In [9]:
def cleanpunctuation(s):
    for p in '!,.:;?':
        s=s.replace(' '+p,p)
    s=s.replace(' '+'n\'t','n\'t')
    s=s.replace(' '+'\'s','\'s')
    s=s.replace(' '+'\'re','\'re')
    s=s.replace(' '+'\'ve','\'ve')
    s=s.replace(' '+'\'ll','\'ll')
    s=s.replace(' '+'\'am','\'am')
    s=s.replace(' '+'\'m','\'m')
    s=s.replace(' '+'\' m','\'m')
    s=s.replace(' '+'\'m','\'m')
    s=s.replace(' '+'\' ve','\'ve')
    s=s.replace(' '+'\' s','\'s')
    s=s.replace('<newline>','\n')
    return s   

In [10]:
train_text=combinetext('train.wp_source', 'train.wp_target')
train_text=list(map(cleanpunctuation,train_text))

In [11]:
valid_text=combinetext('valid.wp_source', 'valid.wp_target')
valid_text=list(map(cleanpunctuation,valid_text))

In [12]:
test_text=combinetext('test.wp_source', 'test.wp_target')
test_text=list(map(cleanpunctuation,valid_text))

Primjer kombinirane priče i prompta 

In [13]:
train_text[10]

"[ WP ] Season 30 of Game of Thrones <sep> Note: I just watched Episode 1 of Season 4 and haven't read any of the books, so I don't know what happens after that. Please no spoilers! This is also my first WP post, but constructive feedback is welcome. Also, just for fun I tried to do an unofficial screenplay format, hence the weird capitalization. \n \n -- - \n \n Fade in to a mound of swords, tips facing us. We hear footsteps on stone, slowly getting closer. Meanwhile, the camera zooms out until we recognize this as The Iron Throne. Camera sweeps around the throne, until we are in front of middle-aged DAENARYS sitting on the throne, face blank. \n \n After several seconds, in addition to the footsteps we now also hear the jangle of a maester's chain. Both sounds then stop, and we hear an aged but familiar voice, `` Your Grace... '' \n \n The camera turns to face SAM, in full maester garb. We see him approaching the camera for a few seconds, and then switches to a side view as he stops 

## 3. Tokenizacija i priprema podataka za Trainer API

GPT-Neo koristi isti tokenizer kao i GPT-2, Byte-Pair Encoding (BPE) za tokenizaciju niza teksta. BPE počinje s vokabularom pojedinačnih znakova i iterativno spaja najčešće parove znakova u nove podtokene, efikasno kodirajući strukturu teksta.

Kako bih osigurao da se nizovi u istoj seriji podataka imaju istu dužinu, postavljam maksimalnu dužinu niza na 512, skraćujem duže nizove i nadopunjavam kraće nizove. Pošto funkcija tokenizer vraća samo input_ids i attention_mask, za potrebe obuke moram pružiti oznake (ciljeve) modelu. Zato stvaram niz oznaka za svaki niz input_ids.

In [2]:
from transformers import AutoTokenizer, DataCollatorWithPadding,GPTNeoForCausalLM, GPT2Tokenizer

In [3]:
model = GPTNeoForCausalLM.from_pretrained("Tincando/fiction_story_generator")
tokenizer = GPT2Tokenizer.from_pretrained("Tincando/fiction_story_generator")

In [12]:
tokenizer.pad_token=tokenizer.eos_token

inputs_train = tokenizer(train_text, padding=True,truncation=True,max_length=512)

In [13]:
inputs_valid=tokenizer(valid_text, padding=True,truncation=True,max_length=512)

In [14]:
def create_labels(inputs):
    labels=[]
    for ids,attention_mask in zip(inputs['input_ids'],inputs['attention_mask']):
        label=ids.copy()
        real_len=sum(attention_mask)
        padding_len=len(attention_mask)-sum(attention_mask)
        label[:]=label[:real_len]+[-100]*padding_len
        labels.append(label)
    inputs['labels']=labels

In [15]:
create_labels(inputs_train)

In [16]:
create_labels(inputs_valid)

In [17]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [18]:
traindata=Dataset(inputs_train)

In [19]:
validdata=Dataset(inputs_valid)

## 4. Fino podešavanje modela

Broj uzoraka za obuku je 170 375. S jednom GPU na Paperspace-u za obuku modela, trebalo mi je nekoliko dana da potpuno obučim model zbog automatskog gašenja Paperspace-a nakon 6 sati.

In [20]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="fiction_story_generator",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=5,
    push_to_hub=True,
    resume_from_checkpoint=True
)

In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=traindata,
    eval_dataset=validdata,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/notebooks/fiction_story_generator is already a clone of https://huggingface.co/Tincando/fiction_story_generator. Make sure you pull the latest changes with `repo.git_pull()`.


In [25]:
trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/27700 [00:00<?, ?it/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
5,2.851700,3.135746


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=170375, training_loss=0.10689012991218819, metrics={'train_runtime': 8779.3565, 'train_samples_per_second': 155.251, 'train_steps_per_second': 19.406, 'total_flos': 3.56025273679872e+17, 'train_loss': 0.10689012991218819, 'epoch': 5.0})

In [26]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/526M [00:00<?, ?B/s]

Upload file runs/Jul26_11-34-44_nz4ahzqyb6/events.out.tfevents.1690371380.nz4ahzqyb6.32.0:   0%|          | 1.…

To https://huggingface.co/Tincando/fiction_story_generator
   be2ee5e..dd2df5c  main -> main

To https://huggingface.co/Tincando/fiction_story_generator
   dd2df5c..4a5ccd1  main -> main



'https://huggingface.co/Tincando/fiction_story_generator/commit/dd2df5cce2b1ad5d83b7f80c26273e53890132ff'

### 4.1. Evaluacija prije finog podešavanja modela

Pomoću Hugging Face Transformers paketa, možemo lako preuzeti istrenirani GPT-neo model.

Prosječna perpleksnost za validacijski skup podataka prije finog podešavanja iznosi 37.28. Pogledat ćemo perpleksnost i nakon finog podešavanja

In [16]:
model1 = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model1,
    train_dataset=traindata,
    eval_dataset=validdata,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [23]:
import math

eval_results = trainer.evaluate()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
print(f"The average perplexity for valid dataset before fine-tuning is:  {math.exp(eval_results['eval_loss']):.2f}")

The average perplexity for valid dataset before fine-tuning is:  37.28


## 5. Generirajmo priče

Izaberemo prompt iz validacijskog skupa podataka, unesemo ga u model i tražimo model da generira priču koja se proteže na 300 riječi. Rezultat su priče koje pokazuju dobru kvalitetu koristeći ugrađenu generate metodu modela, koja nudi različite opcije dekodiranja kao što su greedy decoding, beam-search decoding i nekoliko tehnika uzorkovanja, uključujući uzorkovanje na osnovu temperature i top-k.

Objašnjenja za korištene parametre su sljedeća:

1. **do_sample**: Kada je postavljeno na False, model koristi greedy decoding.
2. **temperature**: Ovo se odnosi na vrijednost koja prilagođava ili utječe na vjerojatnost sljedećeg tokena u sekvenci.
3. **top_k**: Određuje koliko vokabularnih tokena sa najvećim vjerojatnoćama će biti zadržano kroz proces filtriranja poznat kao top-k filtriranje.
4. **top_p**: Ako je postavljeno na float < 1, samo će se najmanja grupa najvjerojatnijih tokena, čije se pojedinačne vjerojatnoće zajedno iznose na vrijednost veću ili jednaku 'top_p', zadržati za generiranje teksta. To nam omogućava da filtriramo i zadržimo podskup tokena na osnovu njihovih vjerojatnoća kako bismo kontrolirali proces generiranja.
5. **repetition_penalty**:Kada je postavljeno na 1.0, to znači da nema kazne za ponavljanje riječi ili tokena u generiranom tekstu. Međutim, ako postavimo vrijednost različitu od 1.0, uvest će se kazna za ponavljajuće riječi, sprječavajući model od generiranja teksta sa previše ponavljanja.
6. **num_return_sequences**: Kontrolira koliko različitih izlaznih sekvenci će model proizvesti za svaki unos u seriji podataka. Ako je postavljeno na 1 , dobijemo jednu sekvencu po unosu; ako je postavljeno na veću vrijednost, dobijemo više sekvenci po unosu, svaka predstavlja različit potencijalni nastavak ili odgovor.


In [19]:
prompt=valid_text[200][:valid_text[200].find('<sep>')]
target=valid_text[200][valid_text[200].find('<sep>')+5:]

input_ids = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids

def generate_story(prompt,target,model):
    print("====PROMPT====\n")
    print(prompt+"\n")
    print('====CORRESPONDING STORY===\n')
    print(target+"\n")
    input_ids = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids
    
    
   
    gen_tokens = model.generate(
        input_ids,
        max_length=300,
        temperature=0.9,
        top_k=2,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=2
    )
    
    
    for generated_sequence_idx, generated_sequence in enumerate(gen_tokens):
        print("=== GENERATED STORY {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()
        # Decode text
        gen_text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        # Remove all text after eos token
        gen_text = gen_text[: gen_text.find(tokenizer.eos_token)]
        print(gen_text)

generate_story(prompt,target,model1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


====PROMPT====

[ WP ] Every person in the world develops a weird mutation/power the day they turn 16. Everyone's powers are always different, some more insignificant than others. You turn 16, and watch as all your friends discover their newfound ability's. That is, until you discover the severity of your own. 

====CORRESPONDING STORY===

 The funeral was the saddest day of my life. I watched as Cameron's mother and father wept over a closed casket. No parent should ever have to bury their child. That is an agony I wouldn't wish on my worst enemy. They tried their best to avoid giving me terrible looks between the tears, but who could blame them? I had just killed their son. 
 
 Days in the summer had become pretty routine. Wake up, eat breakfast, long run, jump in the pool, and then the fun part, go with a few friends to the sparring gym. Mixed martial arts had become the new craze amongst teenagers since medical technology had now made injuries basically nonexistent. I trained 6 day

Sada ćemo koristiti model koji je fino podešen kako bismo generirali priče s istim promptom koji je korišten prije finog podešavanja.

In [20]:
generate_story(prompt,target,model)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


====PROMPT====

[ WP ] Every person in the world develops a weird mutation/power the day they turn 16. Everyone's powers are always different, some more insignificant than others. You turn 16, and watch as all your friends discover their newfound ability's. That is, until you discover the severity of your own. 

====CORRESPONDING STORY===

 The funeral was the saddest day of my life. I watched as Cameron's mother and father wept over a closed casket. No parent should ever have to bury their child. That is an agony I wouldn't wish on my worst enemy. They tried their best to avoid giving me terrible looks between the tears, but who could blame them? I had just killed their son. 
 
 Days in the summer had become pretty routine. Wake up, eat breakfast, long run, jump in the pool, and then the fun part, go with a few friends to the sparring gym. Mixed martial arts had become the new craze amongst teenagers since medical technology had now made injuries basically nonexistent. I trained 6 day

## 6.Evaluacija nakon finog podešavanja modela

Nakon finog podešavanja, možemo vidjeti da je perpleksnost za validacijski skup podataka otprilike 23, što je znatno bolji rezultat nego prije finog podešavanja.

In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    train_dataset=traindata,
    eval_dataset=validdata,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [28]:
eval_results = trainer.evaluate()

In [29]:
print(f"The average perplexity for valid dataset after fine-tuning is:  {math.exp(eval_results['eval_loss']):.2f}")

The average perplexity for valid dataset after fine-tuning is:  23.01


## 7.Zaključak

Kako sam započeo ovaj projekt, preuzeo sam uzbudljiv izazov finog podešavanja GPT-Neo modela koristeći posebno odabrani skup podataka ispunjen zanimljivim pričama iz fikcije. Rezultati su bili zanimljivi, jer su pokazali jasno poboljšanje perpleksnosti, što znači da je razumijevanje teksta modela značajno napredovalo. Još zanimljivije je da, kada sam procijenio priče generirane fino podešenim modelom, primijetio da je i kvaliteta znatno porasla.

Međutim, kao student, teško je ne uvidjeti da generativno modeliranje jezika predstavlja izuzetno kompleksan posao. Rastuća razlika između ljudske kreativnosti i sposobnosti umjetne inteligencije postaje sve očitija. Naša vještina oblikovanja priča, prenošenje emocija i prilagođavanje različitim stilovima pisanja dokaz su kompleksnosti ljudskog jezika.

Iako postižemo impresivan napredak, moramo shvatiti da smo i dalje daleko od kopiranja dubine i bogatstva ljudskog jezika.

**GPT-Neo** je koristan alat, ali nije čarobnjak kada je riječ o pisanju cjelovitih priča od 300 riječi. Može biti koristan za prevladavanje blokade pisanja i poticanje kreativnosti, ali nedostaje mu potpuna automatizacija generiranja fikcijskih priča.

Međutim, nemojte očekivati da će GPT-Neo obaviti sve teške zadatke. Nije sposoban stvoriti potpunu priču od početka do kraja s dubinom i emocijama koje može pružiti ljudski pisac.

Umjesto toga, koristite **GPT-Neo** kao pomoćnika u pisanju.
To je koristan alat za pisce, ali nije zamjena za ljudsku maštovitost i vještine pripovijedanja.
Dakle, iako može biti koristan dio vašeg procesa pisanja, nije prečac do potpunog automatiziranog pisanja fikcije.

Također sam pokušao stvoriti nove priče koristeći trenutačne promptove s iste Reddit stranice [ WritingPrompts forum](https://www.reddit.com/r/WritingPrompts/), a ovo su rezultati.

---
 ***Example Prompt 1*** : You traveled a year into the future and saw the Earth devoid of life. You wanted to find out what happened and went back day by day, over a 1000 days and there are still no signs of life.
```


`` What is this? '' 
 
 I looked up from my book. It was a book I had been reading for years now. A book that had been my home for the past 3 years. My father had always told me that it was a book he had read when he was a child, but I never understood. The pages were black and the words were black. 
 
 `` This is the future. We have been here for 1000 days and nothing has changed. There is no sign of life in the world. But we have to go forward. If we don't we will be destroyed. So we must go forward. '' 
 
 I stood there for a few seconds, confused as to why I was still here. I looked around. Everything was black. No sign of life. Nothing. Just a blank white room with nothing. 
 
 Then I saw it. A small, black object. It was a large, black box. I could not tell where the box ended or what it was made of. I looked closer. It was a large, black cube. I looked again. It was made of metal, and it was covered in dust.
 ```
---

---
 ***Example Prompt 2*** : 'Need a hand?', you nod, and watch in horror as your friend lends you a still-warm, still-twitching, perfectly amputated hand from their pocket.
```

I'm not sure if this is what I wanted to write, or if it's the only thing I've been able to get out of my life for. 
 
 It was a normal day at work when I received an email that said `` Need help with your new prosthetic? '' I looked up from the computer screen to see my boss, Mr. Smith, sitting on his couch, reading a book about prosthetics and how they could be used to replace people who have lost their limbs. He had a clipboard with him and a pen, and he was scribbling away. 
 
 `` What do you mean, 'need a hand '? '' I asked, trying to sound as inconspicuous as possible. 
 
 `` Well, we're all about prosthetics right now. We can't use them to replace people who have lost their limbs, because there's no other way around it. So we need to make sure we don't lose our hands. And we need to keep them warm so they can't die. That's why we're going to make sure that they stay warm, and that they're still alive. You'll have to find a replacement. '' 
 
 The man on the couch looked at me
 
```
---

---
 ***Example Prompt 3*** : You are cooking lunch in the kitchen when you suddenly realize that you have misplaced your kitchen knife. Luckily, when you turn around, your teddy bear is holding a knife and approaching you.
```

 `` I'm sorry, but I don't think this is what I wanted. '' 
 
 The knife was still in his hand, but it had already begun to move. It had been a long day at work, and he was tired of working for himself. He was tired of the boss, and the constant reminders that he was a failure. His boss had been the same as always, but he was different. He was different from everyone else, and he had to make sure that everyone was right. But he was not. 
 
 He had to make sure that he was right. And so, he began to make his way towards the kitchen. He had to make sure that the teddy bear was holding the knife. That was the first thing he had to do. 
 
 As he approached, he noticed something that he had never seen before. A small, round object. It was a small, wooden box. It was made out of a plastic, and it had been placed on the floor. There was a large hole on the bottom of the box, and it was covered in a thick layer of dirt. 
 
 He looked around, and found that he was in a large, wooden room with a larg
 
```
---

---
 ***Example Prompt 4*** : After being eaten by a monstrous whale, you sing to pass the time; unknowingly, your voice becomes a Siren, drawing in doomed sailors.
```

 `` I'm sorry, '' I said. 
 
 She looked up at me, her eyes wide and her face pale. 
 
 `` It's okay. We're not gon na be able to get back to the ship again, '' she said, her voice cracking. 
 
 I looked down at my feet, and I saw the ocean. The waves crashed against the rocks, crashing into the shoreline. A few of the sailors were still alive, and I could hear the screams of the sailors. They were screaming for help. 
 
 `` What are you doing? '' she asked. Her voice was soft and soothing. 
 
 `` Just singing. You can't hear us. '' 
 
 I looked at her, and I saw the sea. It was a sea that was not mine. It was a sea of fire and pain. 
 
 `` Why did you do this to us? Do we have to die? '' 
 
 `` No, no, it doesn't matter. This is the only thing that matters. '' 
 
 I looked at the ocean again, and I saw the ocean. The waves crashed against the rocks, crashing into the shoreline. A few of the sailors were still aliv
 
```
---